# Prediction of housing prices using Random Forest Regression

## Importing libraries

In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

## Reading training set of data

In [2]:
TRAINING_SET_PATH = '../dataset/train.csv'
training_set = pd.read_csv(TRAINING_SET_PATH)
training_set.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## Data Preprocessing

### Handling missing values

In [3]:
def display_missing_values(df):
    missing_values = df.isnull().sum().to_frame()
    missing_values.columns = ['Missing_Values']
    missing_values = missing_values.query('Missing_Values > 0')
    print(missing_values)

display_missing_values(training_set)

              Missing_Values
LotFrontage              259
Alley                   1369
MasVnrType               872
MasVnrArea                 8
BsmtQual                  37
BsmtCond                  37
BsmtExposure              38
BsmtFinType1              37
BsmtFinType2              38
Electrical                 1
FireplaceQu              690
GarageType                81
GarageYrBlt               81
GarageFinish              81
GarageQual                81
GarageCond                81
PoolQC                  1453
Fence                   1179
MiscFeature             1406


#### Dropping outlier rows

Observed in the above cell output, columns *MasVnrArea* and *Electrical* have very low numbers of missing values within.

It is safe to drop the corresponding rows without having a significant impact to the model.

In [4]:
cleaned_set = training_set
cleaned_set.dropna(axis=0, subset=['MasVnrArea', 'Electrical'], inplace=True)
display_missing_values(cleaned_set)

              Missing_Values
LotFrontage              257
Alley                   1361
MasVnrType               863
BsmtQual                  37
BsmtCond                  37
BsmtExposure              38
BsmtFinType1              37
BsmtFinType2              38
FireplaceQu              685
GarageType                81
GarageYrBlt               81
GarageFinish              81
GarageQual                81
GarageCond                81
PoolQC                  1444
Fence                   1170
MiscFeature             1397


#### Handling Bsmt

Upon further inspection in the dataset, rows with NA *BsmtQual* have NA values for the rest of *BsmtColumns*

It would be suitable to create a new column of binary value to indicate whether the row has a *Bsmt*

In [5]:
# Inspecting rows with NA BsmtQual
mask = cleaned_set['BsmtQual'].isna()
cols = ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2')
cleaned_set.loc[mask, cols]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2
17,NaN,NaN,NaN,NaN,NaN
39,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN
156,NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,NaN,NaN,NaN
259,NaN,NaN,NaN,NaN,NaN
342,NaN,NaN,NaN,NaN,NaN
362,NaN,NaN,NaN,NaN,NaN
371,NaN,NaN,NaN,NaN,NaN


In [11]:
# Inspecting rows with non-NA BsmtQual
mask = cleaned_set['BsmtQual'].notna()
cols = ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2')
cleaned_set.loc[mask, cols]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2
0,Gd,TA,No,GLQ,Unf
1,Gd,TA,Gd,ALQ,Unf
2,Gd,TA,Mn,GLQ,Unf
3,TA,Gd,No,ALQ,Unf
4,Gd,TA,Av,GLQ,Unf
...,...,...,...,...,...
1455,Gd,TA,No,Unf,Unf
1456,Gd,TA,No,ALQ,Rec
1457,TA,Gd,No,GLQ,Unf
1458,TA,TA,Mn,GLQ,Rec


In [7]:
# Creating a indicator column
cleaned_set = cleaned_set.assign(BsmtIncluded=training_set['BsmtQual'].notna())

# Replaceing NA with an arbitrary value
mask = cleaned_set['BsmtQual'].isna()
cols = ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2')
cleaned_set.loc[mask, cols] = 'NA'  # Since the features are categorical (str), the value will be as such

display_missing_values(cleaned_set)

              Missing_Values
LotFrontage              257
Alley                   1361
MasVnrType               863
BsmtExposure               1
BsmtFinType2               1
FireplaceQu              685
GarageType                81
GarageYrBlt               81
GarageFinish              81
GarageQual                81
GarageCond                81
PoolQC                  1444
Fence                   1170
MiscFeature             1397


Since there are only 1 missing value left for *BsmtExposure* and *BsmtFinType2*, the corresponding row can be dropped.

In [8]:
cleaned_set = cleaned_set.dropna(axis=0, subset=['BsmtExposure', 'BsmtFinType2'])
display_missing_values(cleaned_set)

              Missing_Values
LotFrontage              257
Alley                   1359
MasVnrType               863
FireplaceQu              685
GarageType                81
GarageYrBlt               81
GarageFinish              81
GarageQual                81
GarageCond                81
PoolQC                  1442
Fence                   1168
MiscFeature             1395


#### Handling Garage

Upon further inspection in the dataset, rows with NA *GarageType* have NA values for the rest of *GarageColumns*

It would be suitable to create a new column of binary value to indicate whether the row has a garage

In [9]:
# Inspecting rows with NA GarageType
mask = cleaned_set['GarageType'].isna()
cols = ('GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond')
cleaned_set.loc[mask, cols]

,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond
39,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN
78,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1349,NaN,NaN,NaN,NaN,NaN
1407,NaN,NaN,NaN,NaN,NaN
1449,NaN,NaN,NaN,NaN,NaN
1450,NaN,NaN,NaN,NaN,NaN


In [10]:
# Inspecting rows with non-NA GarageType
mask = cleaned_set['GarageType'].notna()
cols = ('GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond')
cleaned_set.loc[mask, cols]

,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond
0,Attchd,2003.0,RFn,TA,TA
1,Attchd,1976.0,RFn,TA,TA
2,Attchd,2001.0,RFn,TA,TA
3,Detchd,1998.0,Unf,TA,TA
4,Attchd,2000.0,RFn,TA,TA
...,...,...,...,...,...
1455,Attchd,1999.0,RFn,TA,TA
1456,Attchd,1978.0,Unf,TA,TA
1457,Attchd,1941.0,RFn,TA,TA
1458,Attchd,1950.0,Unf,TA,TA


In [12]:
# Creating a indicator column
cleaned_set = cleaned_set.assign(GarageIncluded=training_set['GarageType'].notna())

# Replaceing NA with an arbitrary value
mask = cleaned_set['GarageType'].isna()

# Handling categorical features
cat_cols = ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond')
cleaned_set.loc[mask, cat_cols] = 'NA'

# Handling numerical feature
num_cols = ('GarageYrBlt')
cleaned_set.loc[mask, num_cols] = 0

display_missing_values(cleaned_set)

             Missing_Values
LotFrontage             257
Alley                  1359
MasVnrType              863
FireplaceQu             685
PoolQC                 1442
Fence                  1168
MiscFeature            1395


#### Handling Alley

It is observed that *Alley* is a nominal feature. NAs in *Alley* are replaced with *'No_Alley'* for now, which will be further processed and encoded.

In [13]:
cleaned_set['Alley'].unique()

array([nan, 'Grvl', 'Pave'], dtype=object)

In [14]:
# Creating a indicator column
cleaned_set = cleaned_set.assign(AlleyIncluded=training_set['Alley'].notna())

mask = cleaned_set['Alley'].isna()
cleaned_set.loc[mask, 'Alley'] = 'No_Alley'
display_missing_values(cleaned_set)

             Missing_Values
LotFrontage             257
MasVnrType              863
FireplaceQu             685
PoolQC                 1442
Fence                  1168
MiscFeature            1395


#### Handling MasVnrType

Again, *MasVnrType* is a nominal feature, NAs are replaced with *'No_MasVnrType'* for further encoding.

In [15]:
cleaned_set['MasVnrType'].unique()

array(['BrkFace', nan, 'Stone', 'BrkCmn'], dtype=object)

In [16]:
# Creating a indicator column
cleaned_set = cleaned_set.assign(MasVnrIncluded=training_set['MasVnrType'].notna())

mask = cleaned_set['MasVnrType'].isna()
cleaned_set.loc[mask, 'MasVnrType'] = 'No_MasVnrType'
display_missing_values(cleaned_set)

             Missing_Values
LotFrontage             257
FireplaceQu             685
PoolQC                 1442
Fence                  1168
MiscFeature            1395


#### Handling FireplaceQu

Same approach for *FireplaceQu*

In [17]:
cleaned_set['FireplaceQu'].unique()

array([nan, 'TA', 'Gd', 'Fa', 'Ex', 'Po'], dtype=object)

In [18]:
# Creating a indicator column
cleaned_set = cleaned_set.assign(FireplaceIncluded=training_set['FireplaceQu'].notna())

mask = cleaned_set['FireplaceQu'].isna()
cleaned_set.loc[mask, 'FireplaceQu'] = 'No_FireplaceQu'
display_missing_values(cleaned_set)

             Missing_Values
LotFrontage             257
PoolQC                 1442
Fence                  1168
MiscFeature            1395


#### Handling PoolQC

Same approach for *PoolQC*

In [19]:
cleaned_set['PoolQC'].unique()

array([nan, 'Ex', 'Fa', 'Gd'], dtype=object)

In [20]:
# Creating a indicator column
cleaned_set = cleaned_set.assign(PoolIncluded=training_set['PoolQC'].notna())

mask = cleaned_set['PoolQC'].isna()
cleaned_set.loc[mask, 'PoolQC'] = 'No_PoolQC'
display_missing_values(cleaned_set)

             Missing_Values
LotFrontage             257
Fence                  1168
MiscFeature            1395


#### Handling Fence

Same approach for *Fence*

In [21]:
cleaned_set['Fence'].unique()

array([nan, 'MnPrv', 'GdWo', 'GdPrv', 'MnWw'], dtype=object)

In [22]:
# Creating a indicator column
cleaned_set = cleaned_set.assign(FenceIncluded=training_set['Fence'].notna())

mask = cleaned_set['Fence'].isna()
cleaned_set.loc[mask, 'Fence'] = 'No_Fence'
display_missing_values(cleaned_set)

             Missing_Values
LotFrontage             257
MiscFeature            1395


#### Handling MiscFeature

Same approach for *MiscFeature*

In [23]:
cleaned_set['MiscFeature'].unique()

array([nan, 'Shed', 'Gar2', 'Othr', 'TenC'], dtype=object)

In [24]:
# Creating a indicator column
cleaned_set = cleaned_set.assign(MiscFeatureIncluded=training_set['MiscFeature'].notna())

mask = cleaned_set['MiscFeature'].isna()
cleaned_set.loc[mask, 'MiscFeature'] = 'No_MiscFeature'
display_missing_values(cleaned_set)

             Missing_Values
LotFrontage             257


#### Handling LotFrontage

Since not all housing properties have lot frontage, such as apartments, it would be better to preserve the implication in the original dataset, instead of making up an approximate value.

In [25]:
mask = cleaned_set['LotFrontage'].isna()
cleaned_set.loc[mask, 'LotFrontage'] = 0
display_missing_values(cleaned_set)

Empty DataFrame
Columns: [Missing_Values]
Index: []


### Encoding categorical features

#### Getting categorical columns

In [27]:
cat_cols = cleaned_set.select_dtypes(include=['object']).columns
cat_cols

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

#### Encoding categorical columns using OrdinalEncoder

It is noted that not all categorical columns are ordinal features.

Despite the added implication, it doesn't really matter that nominal values are treated as such.

In [37]:
ord_enc = OrdinalEncoder()
ord_enc.fit(cleaned_set.loc[:, cat_cols])
encoded_cat = ord_enc.transform(cleaned_set.loc[:, cat_cols])
encoded_cat

array([[3., 1., 1., ..., 1., 8., 4.],
       [3., 1., 1., ..., 1., 8., 4.],
       [3., 1., 1., ..., 1., 8., 4.],
       ...,
       [3., 1., 1., ..., 3., 8., 4.],
       [3., 1., 1., ..., 1., 8., 4.],
       [3., 1., 1., ..., 1., 8., 4.]], shape=(1449, 43))

#### Decoding categorical columns

In [38]:
ord_enc.inverse_transform(encoded_cat)

array([['RL', 'Pave', 'No_Alley', ..., 'No_MiscFeature', 'WD', 'Normal'],
       ['RL', 'Pave', 'No_Alley', ..., 'No_MiscFeature', 'WD', 'Normal'],
       ['RL', 'Pave', 'No_Alley', ..., 'No_MiscFeature', 'WD', 'Normal'],
       ...,
       ['RL', 'Pave', 'No_Alley', ..., 'Shed', 'WD', 'Normal'],
       ['RL', 'Pave', 'No_Alley', ..., 'No_MiscFeature', 'WD', 'Normal'],
       ['RL', 'Pave', 'No_Alley', ..., 'No_MiscFeature', 'WD', 'Normal']],
      shape=(1449, 43), dtype=object)

#### Subtituting encoded categorical features into the dataset

In [43]:
cleaned_set.loc[:, cat_cols] = encoded_cat
cleaned_set.loc[:, cat_cols]

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,3.0,1.0,1.0,3.0,3.0,0.0,4.0,0.0,5.0,2.0,...,1.0,2.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,4.0
1,3.0,1.0,1.0,3.0,3.0,0.0,2.0,0.0,24.0,1.0,...,1.0,2.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,4.0
2,3.0,1.0,1.0,0.0,3.0,0.0,4.0,0.0,5.0,2.0,...,1.0,2.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,4.0
3,3.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,6.0,2.0,...,5.0,3.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,0.0
4,3.0,1.0,1.0,0.0,3.0,0.0,2.0,0.0,15.0,2.0,...,1.0,2.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,3.0,1.0,1.0,3.0,3.0,0.0,4.0,0.0,8.0,2.0,...,1.0,2.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,4.0
1456,3.0,1.0,1.0,3.0,3.0,0.0,4.0,0.0,14.0,2.0,...,1.0,3.0,5.0,5.0,2.0,3.0,2.0,1.0,8.0,4.0
1457,3.0,1.0,1.0,3.0,3.0,0.0,4.0,0.0,6.0,2.0,...,1.0,2.0,5.0,5.0,2.0,3.0,0.0,3.0,8.0,4.0
1458,3.0,1.0,1.0,3.0,3.0,0.0,4.0,0.0,12.0,2.0,...,1.0,3.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,4.0
